In [1]:
import json
import requests
import time
from tqdm import tqdm

API_KEY = "2a2299dd95944956b69397a89113d5a7.GW5jR7NYhANOuGES"  # 替换为你的 API Key
API_URL = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

def build_prompt(question, answer):
    return f"""
你是一位保险领域的资深专家，请对以下问答对进行评分，满分为5分。

【问题】：{question}
【回答】：{answer}

请你评估以下几个方面：
1. 回答是否准确反映文档内容（准确性）；
2. 回答是否完整涵盖要点（完整性）；
3. 回答是否清晰易懂（清晰度）；

请按以下格式回答：
分数：X.X
评语：......
"""

def score_qa_pair(question, answer):
    prompt = build_prompt(question, answer)
    data = {
        "model": "glm-4",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.2
    }
    try:
        response = requests.post(API_URL, headers=HEADERS, json=data)
        if response.status_code == 200:
            result = response.json()
            return result["choices"][0]["message"]["content"]
        else:
            return f"请求失败: {response.status_code} - {response.text}"
    except Exception as e:
        return f"异常: {str(e)}"

def process_testset(input_file, output_file):
    with open(input_file, "r", encoding="utf-8") as f:
        qa_list = json.load(f)

    results = []
    for item in tqdm(qa_list, desc="评分进度", unit="对"):
        question = item.get("question", "")
        answer = item.get("answer", "")
        feedback = score_qa_pair(question, answer)
        item["score_feedback"] = feedback
        results.append(item)
        time.sleep(2) 

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    print("所有问答评分已完成，结果保存在：", output_file)

process_testset(r"D:\desktop\code\QA\train.json", "qa_scored.json")

评分进度: 100%|██████████| 97/97 [12:11<00:00,  7.54s/对]

所有问答评分已完成，结果保存在： qa_scored.json
